In [11]:
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy as sp

In [12]:
mic_coords = pd.read_excel("data/mic_coords_final.xlsx", index_col=0)
distances = pd.read_excel("data/miscellaneous_positions.xlsx", index_col=0)


def get_mic_coords(name, df, x, y, z):
    filtered_df = df[df['Mic_Index'] == name]
    coordinates = filtered_df[[x, y, z]].to_numpy()
    return coordinates[0]



position_coords_default = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X", "Y", "Z")
    position_coords_default.append(coord)
position_coords_default = np.array(position_coords_default)

position_coords_optimized = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X_optimized", "Y_optimized", "Z_optimized")
    position_coords_optimized.append(coord)
position_coords_optimized = np.array(position_coords_optimized)

distances_array = distances["distance"].to_numpy()

In [13]:
#from scipy.optimize import least_squares

def trilaterate(ref_points, distances):
    """
    Solves the 3D trilateration problem using least squares optimization.
    
    Parameters:
    - ref_points: List of N (x, y, z) coordinates of reference points (N >= 4)
    - distances: List of N distances from each reference point to the unknown point
    
    Returns:
    - (x, y, z): Estimated coordinates of the unknown point
    """
    
    # Initial guess: Average of reference points
    initial_guess = np.mean(ref_points, axis=0)

    # Function to minimize: Distance equation error for all reference points
    def error_function(unknown):
        return np.linalg.norm(ref_points - unknown, axis=1) - distances

    # Solve using least squares optimization
    result = sp.optimize.least_squares(error_function, initial_guess, method='lm')

    return result.x  # Return the estimated (x, y, z) coordinates

In [14]:
bassoon_low = trilaterate(position_coords_optimized[0:6], distances_array[0:6])
print("bassoon_low xyz:", bassoon_low)

bassoon_high = trilaterate(position_coords_optimized[6:12], distances_array[6:12])
print("bassoon_high xyz:", bassoon_high)

camera_1 = trilaterate(position_coords_optimized[12:18], distances_array[12:18])
print("camera_1 xyz:", camera_1)

camera_2 = trilaterate(position_coords_optimized[18:24], distances_array[18:24])
print("camera_2 xyz:", camera_2)


referecene_speaker = trilaterate(position_coords_optimized[24:29], distances_array[24:29])
print("referecene_speaker xyz:", referecene_speaker)

bassoon_low xyz: [0.97081132 0.98296962 0.41316809]
bassoon_high xyz: [0.53056959 0.3204048  1.48546785]
camera_1 xyz: [1.3236992  0.04429161 0.52348324]
camera_2 xyz: [1.16191888 0.76769209 1.82988376]
referecene_speaker xyz: [0.72439712 0.62713966 0.55611907]


In [15]:

fig = px.scatter_3d(mic_coords, x='X_optimized', y='Y_optimized', z='Z_optimized', text='Mic_Index', title='3D Scatter Plot', color='Plane', width=1000, height=1000)


# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[bassoon_low[0]],  # x-coordinate of the single point
    y=[bassoon_low[1]],  # y-coordinate of the single point
    z=[bassoon_low[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='black'),  # Customize the marker style
    text=['bassoon_low'],  # Optional text label for the point
    name='bassoon_low'  # Legend entry
))

# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[bassoon_high[0]],  # x-coordinate of the single point
    y=[bassoon_high[1]],  # y-coordinate of the single point
    z=[bassoon_high[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='black'),  # Customize the marker style
    text=['bassoon_high'],  # Optional text label for the point
    name='bassoon_high'  # Legend entry
))

# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[camera_1[0]],  # x-coordinate of the single point
    y=[camera_1[1]],  # y-coordinate of the single point
    z=[camera_1[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='black'),  # Customize the marker style
    text=['camera_1'],  # Optional text label for the point
    name='camera_1'  # Legend entry
))

# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[camera_2[0]],  # x-coordinate of the single point
    y=[camera_2[1]],  # y-coordinate of the single point
    z=[camera_2[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='black'),  # Customize the marker style
    text=['camera_2'],  # Optional text label for the point
    name='camera_2'  # Legend entry
))

# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[referecene_speaker[0]],  # x-coordinate of the single point
    y=[referecene_speaker[1]],  # y-coordinate of the single point
    z=[referecene_speaker[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='black'),  # Customize the marker style
    text=['referecene_speaker'],  # Optional text label for the point
    name='referecene_speaker'  # Legend entry
))

fig.update_layout(scene=dict(
    xaxis=dict(range=[1.5, 0]), # x-Achse umdrehen - "linke hand regel"
))

# Plot anzeigen
fig.show()